In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import numpy as np
import os
import pickle
import gc
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import Normalizer, StandardScaler
from utils import *

In [13]:
stock_symbol = '5871.TW'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
end_date = '2024-12-31'

# 擷取股票價格資訊
scaler = StandardScaler()
stock_price_data = fetch_stock_price(stock_symbol=stock_symbol, start_date='2012-01-02',end_date=end_date)

"""
pctchange: (today - yesterday)/yesterday
"""
stock_price_data['do'] = stock_price_data['Open'].pct_change() * 100
stock_price_data['dh'] = stock_price_data['High'].pct_change() * 100
stock_price_data['dl'] = stock_price_data['Low'].pct_change() * 100
stock_price_data['dc'] = stock_price_data['Close'].pct_change() * 100
stock_price_data['dv'] = stock_price_data['Volume'].pct_change() * 100

"""
do_1, dc_1, doc_1: tmr's information
"""
stock_price_data['do_1'] = stock_price_data['do'].shift(-1)
stock_price_data['dc_1'] = stock_price_data['dc'].shift(-1)
stock_price_data['doc_1'] = \
    ((stock_price_data['Close'].shift(-1) - stock_price_data['Open'].shift(-1))/stock_price_data['Open'].shift(-1))\
    *100

stock_price_data = stock_price_data.dropna()

# df = stock_price_data.iloc[:,7:]
df = stock_price_data
# Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df = df.dropna()

# scaling
df['Close_origin'] = df['Close']
scaler = StandardScaler()
scaler.fit(df[['do', 'dh', 'dl', 'dc', 'dv', 'Close']][:2000])
df[['do', 'dh', 'dl', 'dc', 'dv', 'Close']] = scaler.fit_transform(df[['do', 'dh', 'dl', 'dc', 'dv', 'Close']])


/home/jacob/anaconda3/envs/mlntu/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_229809/3308488612.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close_origin'] = df['Close']
/tmp/ipykernel_229809/3308488612.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['do', 'dh', 'd

In [14]:
num_class = 2
batch_size = 32

x, y, date = window_x_y(df, num_class, 100)
# X = process_x(x)
X, x_test, y, y_test = train_test(x, y)
x_train, x_valid, y_train, y_valid = train_valid(X, y)

""" Standard
mean = torch.mean(x_train)
std = torch.std(x_train)
print(std)
x_train = normalize(x_train, mean, std)
mean = torch.mean(x_valid)
std = torch.std(x_valid)
print(std)
x_valid = normalize(x_valid, mean, std)
x_test = normalize(x_test, mean, std)
"""

100%|██████████| 2853/2853 [00:04<00:00, 686.10it/s]


' Standard\nmean = torch.mean(x_train)\nstd = torch.std(x_train)\nprint(std)\nx_train = normalize(x_train, mean, std)\nmean = torch.mean(x_valid)\nstd = torch.std(x_valid)\nprint(std)\nx_valid = normalize(x_valid, mean, std)\nx_test = normalize(x_test, mean, std)\n'

### Double

In [15]:
"""src = use_src(df)
src = torch.tensor(src).to(dtype=torch.float32).unsqueeze(0).repeat(batch_size, 1, 1)
src.shape, x_train.shape"""

(torch.Size([32, 6, 2000]), (2053, 6, 100))

In [16]:

trainloader, validloader, testloader = (
    loader(
        torch.tensor(x_train).to(dtype=torch.float32), 
        torch.tensor(y_train).to(dtype=torch.float32), 
        batch_size=batch_size), 
    loader(
        torch.tensor(x_valid).to(dtype=torch.float32), 
        torch.tensor(y_valid).to(dtype=torch.float32), 
        batch_size=batch_size),
    loader(
        torch.tensor(x_test).to(dtype=torch.float32), 
        torch.tensor(y_test).to(dtype=torch.float32), 
        batch_size=batch_size)
    )    

test_date = df.index[-len(y_test):]

src = use_src(df)
src = torch.tensor(src).to(dtype=torch.float32).unsqueeze(0).repeat(batch_size, 1, 1)
src = torch.tensor(x_train).to(dtype=torch.float32)

if num_class == 1:
    with open('./DataLoader/dataloader_1.pk', 'wb') as f:
        pickle.dump({'trainloader': trainloader, 'validloader': validloader, 'testloader': testloader}, f)
elif num_class == 2:
    with open('./DataLoader/dataloader.pk', 'wb') as f:
        pickle.dump({'trainloader': trainloader, 'validloader': validloader, 'testloader': testloader}, f)
with open('DataLoader/dates.pk', 'wb') as f:
    pickle.dump({'test': test_date}, f)
with open('DataLoader/data_clean.pk', 'wb') as f:
    pickle.dump(df, f)
with open('DataLoader/src.pk', 'wb') as f:
    pickle.dump(src, f)

In [17]:
import pandas as pd
pd.DataFrame(y_test).set_index(df.index[-len(y_test):])

,0,1
Date,,
2022-12-15 00:00:00+08:00,0.233645,-2.277901
2022-12-16 00:00:00+08:00,-0.699299,1.631704
2022-12-19 00:00:00+08:00,1.643188,-2.064226
2022-12-20 00:00:00+08:00,-0.692838,2.810305
2022-12-21 00:00:00+08:00,2.325582,-1.822324
...,...,...
2024-02-22 00:00:00+08:00,-0.566572,0.000000
2024-02-23 00:00:00+08:00,0.284900,-1.704545
2024-02-26 00:00:00+08:00,-1.136364,-0.578035
